In [ ]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
import torch.nn.functional as F
import torch.optim as optim
import time
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import json 
import re

In [ ]:
downloaded = drive.CreateFile({'id': '1R0V2OuTW12oupYMk07RJJFOr-EqpVHTI'})
downloaded.GetContentFile('Tweets.csv')
df_orig = pd.read_csv('Tweets.csv')

downloaded = drive.CreateFile({'id': '1FHE2Zpp7i4I3XnuHqbqKygGh3RcMfJ0J'})
downloaded.GetContentFile('dictionary.json')
  
f = open('dictionary.json',) 
data = json.load(f) 

df_orig.head()

,sentiment,text,processed_text,length,1_word,2_word,3_word,4_word,5_word,6_word,7_word,8_word,9_word,10_word,11_word,12_word,13_word,14_word,15_word,16_word,17_word,18_word,19_word,20_word,21_word,22_word,23_word,24_word,25_word,26_word,27_word,28_word,29_word,30_word
0,0,@VirginAmerica What @dhepburn said.,@virginamerica what @dhepburn said,4,13634,2106,10268,11928,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,@VirginAmerica plus you've added commercials t...,@virginamerica plus youve added commercials to...,9,13634,1715,5248,7030,8653,4463,1903,2752,8770,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,@VirginAmerica I didn't today... Must mean I n...,@virginamerica i didnt today must mean i need ...,12,13634,8075,2858,2834,5694,10500,8075,3457,4463,7816,132,9075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-1,@VirginAmerica it's really aggressive to blast...,@virginamerica its really aggressive to blast ...,17,13634,9067,4484,2406,4463,12531,10701,6923,8404,2958,11955,10016,10936,16644,10010,683,8325,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1,@VirginAmerica and it's a really big bad thing...,@virginamerica and its a really big bad thing ...,10,13634,6149,9067,15211,4484,4383,9808,412,11230,6737,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Data Pre-processing

In [ ]:
cols = ['sentiment', '1_word', '2_word', '3_word', '4_word', '5_word', '6_word',
       '7_word', '8_word', '9_word', '10_word', '11_word', '12_word',
       '13_word', '14_word', '15_word', '16_word', '17_word', '18_word',
       '19_word', '20_word', '21_word', '22_word', '23_word', '24_word',
       '25_word', '26_word', '27_word', '28_word', '29_word', '30_word']
df_seq = pd.DataFrame(columns = cols)

for i in range(df_orig.shape[0]):
  lst=[df_orig.loc[i].sentiment]
  processed_text = re.sub('[^A-Za-z0-9 @]+', '', df_orig.loc[i].text.lower())
  lst.extend([data.get(w) for w in processed_text.split()])
  lst.extend([0] * (31 - len(lst)))
  df_seq = df_seq.append(pd.Series(lst, index = cols), ignore_index=True)
df_seq = df_seq.fillna(0)
df_seq = df_seq.astype('int32')


# EDA

In [ ]:
df_seq.head()

,sentiment,1_word,2_word,3_word,4_word,5_word,6_word,7_word,8_word,9_word,10_word,11_word,12_word,13_word,14_word,15_word,16_word,17_word,18_word,19_word,20_word,21_word,22_word,23_word,24_word,25_word,26_word,27_word,28_word,29_word,30_word
0,0,10348,4290,10815,1667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,10348,10971,6259,13095,9335,8008,11667,8991,7120,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,10348,8598,3623,2391,3090,4707,8598,4805,8008,8757,4718,6022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-1,10348,14276,6398,7987,8008,11742,14871,2463,9054,6700,15758,14352,14887,13292,11658,2436,15493,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1,10348,15348,14276,9717,6398,4289,6623,14735,16131,9329,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_seq.columns

Index(['sentiment', '1_word', '2_word', '3_word', '4_word', '5_word', '6_word',
       '7_word', '8_word', '9_word', '10_word', '11_word', '12_word',
       '13_word', '14_word', '15_word', '16_word', '17_word', '18_word',
       '19_word', '20_word', '21_word', '22_word', '23_word', '24_word',
       '25_word', '26_word', '27_word', '28_word', '29_word', '30_word'],
      dtype='object')

In [ ]:
(tr_unique, tr_counts) = np.unique(df_seq['sentiment'].values, return_counts=True)
perc = tr_counts/np.sum(tr_counts)
pd.DataFrame((tr_counts,perc), columns=tr_unique,index=["Count","Percentage"])

,-1,0,1
Count,9058.000000,3080.000000,2355.000000
Percentage,0.624991,0.212516,0.162492


In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(df_seq.drop(columns = ['sentiment']), df_seq['sentiment']+1, test_size = 0.2, 
                                                    random_state = 0, shuffle = True)

xTr_tensor = torch.Tensor(xTrain.values)
xTs_tensor = torch.Tensor(xTest.values)

yTr_tensor = torch.Tensor(yTrain.values)
yTs_tensor = torch.Tensor(yTest.values)

train = torch.utils.data.TensorDataset(xTr_tensor, yTr_tensor)
train_loaders = torch.utils.data.DataLoader(train, batch_size=100, shuffle=True)
(tr_unique, tr_counts) = np.unique(yTrain.values, return_counts=True)


test = torch.utils.data.TensorDataset(xTs_tensor,yTs_tensor)
test_loaders = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)
(ts_unique, ts_counts) = np.unique(yTest.values, return_counts=True)
print("Pandas: \nTrain Label Unique Coumts Labels: {}, Counts: {} \nTest Label Unique Coumts Labels: {}, Counts: {}  "
          .format(tr_unique, tr_counts,ts_unique,ts_counts))

Pandas: 
Train Label Unique Coumts Labels: [0 1 2], Counts: [7258 2431 1905] 
Test Label Unique Coumts Labels: [0 1 2], Counts: [1800  649  450]  


In [ ]:
# Set up your device 
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")



```
# This is formatted as code
```

# Simple baseline model (using sklearn)

In [ ]:
#simple baseline model (using sklearn)
X = df_seq.loc[:,df_seq.columns!='sentiment']
Y = df_seq['sentiment']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = np.random.seed(42))
st = time.time()
model_binary = LogisticRegression(solver='liblinear')
model_binary.fit(X_train, Y_train)
print(accuracy_score(model_binary.predict(X_test), Y_test)*100)
en = time.time()
print('Total Time to Execute: {:.0f} Seconds'.format(en-st)) 

63.504656778199376
Total Time to Execute: 0 Seconds


# Simple baseline LR model (using pytorch)

In [ ]:
#simple baseline LR model (using pytorch)

X,Y = train_test_split(df_seq, test_size=0.2, random_state = np.random.seed(42))
batch_size=100

train_dataset = TensorDataset( Tensor(X.loc[:,X.columns!='sentiment'].values), Tensor(X['sentiment'].values) )
test_dataset = TensorDataset( Tensor(Y.loc[:,Y.columns!='sentiment'].values), Tensor(Y['sentiment'].values) )

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

n_iters = 3000
epochs = n_iters / (len(train_dataset) / batch_size)
input_dim = 30
output_dim = 3  #{-1, 0, 1}
lr_rate = 0.001

LRmodel = LogisticRegression(input_dim, output_dim)

In [ ]:
# Training model on training data
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(LRmodel.parameters(), lr=lr_rate)
iter = 0
st = time.time()
for epoch in range(int(epochs)):
    for i in enumerate(train_loader):
        inputs = Variable(i[1][0]) #features
        labels = Variable(i[1][1])
        optimizer.zero_grad()
        #print(epoch,inputs,"\n------\n")
        outputs = LRmodel(inputs)
        #print(outputs.squeeze().shape, (labels.long()+1).shape,"\n------\n")
        loss = criterion(outputs.squeeze(), labels.long()+1)
        #print("epoch",epoch,loss)
        loss.backward()
        optimizer.step()

In [ ]:
# Validating on the test data
correct = 0
total = 0
iter_test = 0
for j in enumerate(test_loader):
    iter_test += 1
    inputs = Variable(j[1][0]) #features
    labels = Variable(j[1][1])
    outputs = LRmodel(inputs)
    _, predicted = torch.max(outputs.data, 1)

    # Total number of labels
    total += labels.size(0)

    # Total correct predictions
    correct += (predicted == labels.long()+1).sum()

accuracy = 100 * (correct.item() / total)
print(accuracy)
en = time.time()
print('Total Time to Execute: {:.0f} Seconds'.format(en-st)) 

62.36633321835116
Total Time to Execute: 4 Seconds


# Deep Learning Model - CNN (using pytorch)

In [ ]:
class SimpleCNN(torch.nn.Module):    
    def __init__(self,input_dim, output_dim):
        super(SimpleCNN, self).__init__()
        self.conv1_1 = torch.nn.Conv1d(input_dim, batch_size, kernel_size=2, stride=1, padding=2)
        self.conv1_2 = torch.nn.Conv1d(input_dim, batch_size, kernel_size=1, stride=1)
        self.pool = torch.nn.MaxPool1d(kernel_size=3)
        self.fc1 = torch.nn.Linear(batch_size, output_dim)
        self.fc2 = torch.nn.Linear(output_dim, output_dim)
        
    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        #x1 = F.relu(self.conv1_2(x))
        x = self.pool(x)
        x = x.view(-1, batch_size)
        x = self.fc1(x)
        x = self.fc2(x)
        return(x)

n_iters = 3000
epochs = 50
input_dim = 30
output_dim = 3  #{-1, 0, 1}
lr_rate = 0.01

CNNmodel = SimpleCNN(input_dim, output_dim)

In [ ]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(CNNmodel.parameters(), lr=lr_rate)
st = time.time()
for epoch in range(int(epochs)):
    for i in enumerate(train_loader):
        inputs = Variable(i[1][0]).unsqueeze(2) #features
        labels = Variable(i[1][1])
        optimizer.zero_grad()
        outputs = CNNmodel(inputs)
        loss = criterion(outputs.squeeze(), labels.long()+1)
        loss.backward()
        optimizer.step()

In [ ]:
# Validating on the test data
correct = 0
total = 0
iter_test = 0
total_val_loss = 0
for j in enumerate(test_loader):
    iter_test += 1
    inputs = Variable(j[1][0]).unsqueeze(2) #features
    labels = Variable(j[1][1])
    outputs = CNNmodel(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.long()+1).sum()
accuracy = 100 * (correct.item() / total)
print(accuracy)
en = time.time()
print('Total Time to Execute: {:.0f} Seconds'.format(en-st)) 

64.29803380476025
Total Time to Execute: 21 Seconds


# Train - Test Utils

In [ ]:
# Train and Test Utils.

def train(model, device, train_loader, optimizer, epoch, log_interval = 100, float_dtype=False):
    # Set model to training mode
    model.train()
    num_correct = 0
    # Loop through data points
    for batch_idx, (data, target) in enumerate(train_loader):
    
        # Send data and target to device
        if float_dtype:
            data, target = data.to(device), target.flatten().long().to(device)
            
        else:
            data, target = data.long().to(device), target.flatten().long().to(device)
        
        # Zero out the ortimizer
        optimizer.zero_grad()
        
        # Pass data through model
        output = model(data)
        
        # Compute the negative log likelihood loss
        
        loss = F.nll_loss(output, target)
        
        # Backpropagate loss
        loss.backward()
        
        # Make a step with the optimizer
        optimizer.step()
        
        # Get predictions from the model for each data point
        pred = output.data.max(1, keepdim=True)[1]
        # Add number of correct predictions to total num_correct 
        num_correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
        
        # Print loss 
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        
    """
        # Print loss 
    print('\ Train Accuracy ({:.0f}%)\n'.format(
        100. * num_correct / len(train_loader.dataset)))
    acc = 100. * num_correct / len(train_loader.dataset)
    """
    acc = 100. * num_correct / len(train_loader.dataset)
    return acc

            
# Define test method
def test(model, device, test_loader, float_dtype = False  ):
    # Set model to evaluation mode
    model.eval()
    # Variable for the total loss 
    test_loss = 0
    # Counter for the correct predictions
    num_correct = 0
    
    # don't need autograd for eval
    with torch.no_grad():
        # Loop through data points
        for data, target in test_loader:
        
            # Send data to device
            if float_dtype:
                data, target = data.to(device), target.flatten().long().to(device)
            
            else:
                data, target = data.long().to(device), target.flatten().long().to(device)
            # Pass data through model
            output = model(data) 
            
            # Compute the negative log likelihood loss with reduction='sum' and add to total test_loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            
            # Get predictions from the model for each data point
            pred = output.data.max(1, keepdim=True)[1]
            # Add number of correct predictions to total num_correct 
            num_correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
    # Compute the average test_loss
    avg_test_loss = test_loss /len(test_loader.dataset)
    
    """
    # Print loss 
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))
    acc = 100. * num_correct / len(test_loader.dataset)
    """
    acc = 100. * num_correct / len(test_loader.dataset)
    return acc

# RNN Based Model Definitions

In [ ]:
# Define Models

class FC2Layer(nn.Module):
    def __init__(self, input_size, n_hidden, output_size):
        super(FC2Layer, self).__init__()
        self.input_size = input_size
        self.network = nn.Sequential(
            nn.Linear(input_size, n_hidden), 
            nn.ReLU(), 
            nn.Linear(n_hidden, n_hidden), 
            nn.ReLU(), 
            nn.Linear(n_hidden, output_size), 
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = x.view(-1, self.input_size)
        return self.network(x)
    

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):        
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text.T)
        output, hidden = self.rnn(embedded)
        decoded = self.fc(hidden.squeeze(0))
        
        return F.log_softmax(decoded, dim=1)


class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        embedded = self.dropout(self.embedding(text.T))
        packed_output, (hidden, cell) = self.rnn(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                       
        decoded = self.fc(hidden)
        return  F.log_softmax(decoded, dim=1)

# FCN

In [ ]:
import time
input_size = 30
n_hidden = 100
output_size = 3

# Deifne model and sent to device
model = FC2Layer(input_size, n_hidden, output_size)

model.to(device)

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
tr_acc_list = list()
ts_acc_list = list()
# Training loop with 10 epochs
st = time.time()
for epoch in range(1, 30+ 1):
    
    # Train model
    tr_acc = train(model,device,train_loaders,optimizer,epoch,float_dtype=True)
    tr_acc_list.append(tr_acc)
    
    # Test model
    ts_acc = test(model,device,test_loaders, float_dtype=True)
    ts_acc_list.append(ts_acc)
    
print("******FINAL Accuracy at Last Epoch ******")
print('\nTrain set:  Accuracy: {}, \nTest set:  Accuracy: {}\n'.format(
        tr_acc,ts_acc ))
en = time.time()
print('Total Time to Execute: {:.0f} Seconds'.format(en-st)) 



Train Epoch: 1 [0/11594 (0%)]	Loss: 267.826416
Train Epoch: 1 [10000/11594 (86%)]	Loss: 0.914237
Train Epoch: 2 [0/11594 (0%)]	Loss: 0.852147
Train Epoch: 2 [10000/11594 (86%)]	Loss: 0.952228
Train Epoch: 3 [0/11594 (0%)]	Loss: 0.947160
Train Epoch: 3 [10000/11594 (86%)]	Loss: 0.957642
Train Epoch: 4 [0/11594 (0%)]	Loss: 0.862415
Train Epoch: 4 [10000/11594 (86%)]	Loss: 0.865937
Train Epoch: 5 [0/11594 (0%)]	Loss: 0.893779
Train Epoch: 5 [10000/11594 (86%)]	Loss: 0.824448
Train Epoch: 6 [0/11594 (0%)]	Loss: 0.872060
Train Epoch: 6 [10000/11594 (86%)]	Loss: 0.954461
Train Epoch: 7 [0/11594 (0%)]	Loss: 0.902083
Train Epoch: 7 [10000/11594 (86%)]	Loss: 0.863217
Train Epoch: 8 [0/11594 (0%)]	Loss: 0.815014
Train Epoch: 8 [10000/11594 (86%)]	Loss: 0.914238
Train Epoch: 9 [0/11594 (0%)]	Loss: 0.948782
Train Epoch: 9 [10000/11594 (86%)]	Loss: 0.938451
Train Epoch: 10 [0/11594 (0%)]	Loss: 1.050042
Train Epoch: 10 [10000/11594 (86%)]	Loss: 0.924447
Train Epoch: 11 [0/11594 (0%)]	Loss: 0.923169


# RNN

In [ ]:
input_dim = 17201
embedding_dim = 30
hidden_dim = 512
ouput_dim = 3

model = RNN(input_dim, embedding_dim, hidden_dim, ouput_dim)
model.to(device)

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

tr_acc_list = list()
ts_acc_list = list()
st = time.time()
# Training loop with 10 epochs
for epoch in range(1, 50 + 1):
    
    # Train model
    tr_acc = train(model,device,train_loaders,optimizer,epoch)
    tr_acc_list.append(tr_acc)
    # Test model
    ts_acc = test(model,device,test_loaders)
    ts_acc_list.append(ts_acc)
    print('\nTrain set:  Accuracy: {}, \nTest set:  Accuracy: {}\n'.format(
        tr_acc,ts_acc ))
print("******FINAL Accuracy at Last Epoch ******")
print('\nTrain set:  Accuracy: {}, \nTest set:  Accuracy: {}\n'.format(
        tr_acc,ts_acc ))
en = time.time()
print('Total Time to Execute: {:.0f} Seconds'.format(en-st)) 


Train Epoch: 1 [0/11594 (0%)]	Loss: 1.115612
Train Epoch: 1 [10000/11594 (86%)]	Loss: 0.901312

Train set:  Accuracy: 61.97170950491633, 
Test set:  Accuracy: 62.09037599172128

Train Epoch: 2 [0/11594 (0%)]	Loss: 0.931517
Train Epoch: 2 [10000/11594 (86%)]	Loss: 0.966265

Train set:  Accuracy: 62.60134552354666, 
Test set:  Accuracy: 62.09037599172128

Train Epoch: 3 [0/11594 (0%)]	Loss: 0.996427
Train Epoch: 3 [10000/11594 (86%)]	Loss: 0.842237

Train set:  Accuracy: 62.60134552354666, 
Test set:  Accuracy: 62.09037599172128

Train Epoch: 4 [0/11594 (0%)]	Loss: 0.896809
Train Epoch: 4 [10000/11594 (86%)]	Loss: 0.855384

Train set:  Accuracy: 63.01535276867345, 
Test set:  Accuracy: 64.05657123145913

Train Epoch: 5 [0/11594 (0%)]	Loss: 0.868890
Train Epoch: 5 [10000/11594 (86%)]	Loss: 0.827404

Train set:  Accuracy: 63.53286182508194, 
Test set:  Accuracy: 64.67747499137634

Train Epoch: 6 [0/11594 (0%)]	Loss: 0.782636
Train Epoch: 6 [10000/11594 (86%)]	Loss: 0.924703

Train set:  Ac

# Deep Learning Model - LSTM (using pytorch)

In [ ]:
imput_dim = 17201
embedding_dim = 100
hidden_dim = 256
output_dim = 3
n_layers = 2
dropout = 0.5
bidirectional = True


model = LSTM(imput_dim, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout)
model.to(device)

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
tr_acc_list = list()
ts_acc_list = list()
# Training loop with 30 epochs
for epoch in range(1, 50 + 1):
    
    # Train model
    tr_acc = train(model,device,train_loaders,optimizer,epoch)
    tr_acc_list.append(tr_acc)
    
    # Test model
    ts_acc = test(model,device,test_loaders)
    ts_acc_list.append(ts_acc)
    print('\nTrain set:  Accuracy: {}, \nTest set:  Accuracy: {}\n'.format(
        tr_acc,ts_acc ))
print("******FINAL Accuracy at Last Epoch ******")
print('\nTrain set:  Accuracy: {}, \nTest set:  Accuracy: {}\n'.format(
        tr_acc,ts_acc ))
en = time.time()
print('Total Time to Execute: {:.0f} Seconds'.format(en-st)) 

Train Epoch: 1 [0/11594 (0%)]	Loss: 1.109681
Train Epoch: 1 [10000/11594 (86%)]	Loss: 0.865295

Train set:  Accuracy: 61.178195618423324, 
Test set:  Accuracy: 62.09037599172128

Train Epoch: 2 [0/11594 (0%)]	Loss: 0.896495
Train Epoch: 2 [10000/11594 (86%)]	Loss: 0.912359

Train set:  Accuracy: 62.60134552354666, 
Test set:  Accuracy: 62.09037599172128

Train Epoch: 3 [0/11594 (0%)]	Loss: 1.005753
Train Epoch: 3 [10000/11594 (86%)]	Loss: 0.871718

Train set:  Accuracy: 62.60134552354666, 
Test set:  Accuracy: 62.09037599172128

Train Epoch: 4 [0/11594 (0%)]	Loss: 0.990641
Train Epoch: 4 [10000/11594 (86%)]	Loss: 0.957388

Train set:  Accuracy: 62.60134552354666, 
Test set:  Accuracy: 62.09037599172128

Train Epoch: 5 [0/11594 (0%)]	Loss: 0.899216
Train Epoch: 5 [10000/11594 (86%)]	Loss: 0.941380

Train set:  Accuracy: 62.60134552354666, 
Test set:  Accuracy: 62.09037599172128

Train Epoch: 6 [0/11594 (0%)]	Loss: 0.909003
Train Epoch: 6 [10000/11594 (86%)]	Loss: 0.879427

Train set:  A